In [1]:
import random
import numpy as np
import pandas as pd
from tensorflow import keras
from glob import glob
from skimage.io import imread
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
from commons import plot
from sklearn import metrics
from skimage.transform import resize

2022-05-01 04:08:16.818204: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-01 04:08:16.818222: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def print_evaluate(true, predicted):  
    
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    
def rescale_arr(arr, scale=255):
    return (arr * scale).astype('uint8')

In [3]:
# image | anilha | peso
broilers_weights = pd.read_csv('feature_processing/galinhas_pesos.csv')

In [4]:
from random import randint

def open_images(idx):
    
    folder_images = glob(f'images_cropped/images/{idx}/*')
   
    size_folder = len(folder_images) 
    index_image = randint(0, size_folder)

    image =  folder_images[index_image]
    
    return imread(image)

In [5]:
all_images = list(map(lambda x: x.split('/')[2], glob("images_cropped/images/*")))

In [6]:
images = []
anilhas = []

for index in tqdm(all_images):
    try:
        img_resized = resize(open_images(index), (224, 224))
        img = rescale_arr(img_resized)
        images.append(img)
        anilhas.append(int(index))
    except: 
        continue

100%|██████████████████████████████████████████| 25/25 [00:00<00:00, 529.84it/s]


In [7]:
weights = [None] * len(anilhas)

for anilha, peso in zip(broilers_weights.anilha,  broilers_weights.peso):
    try:
        weights[anilhas.index(anilha)] = peso
    except:
        continue

In [8]:
images = np.asarray(images)
weights = np.asarray(weights)

X_train, X_test, y_train, y_test = train_test_split(images, weights, test_size=0.2)

In [9]:
input_shape = images[0].shape + (1,)

In [10]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        
        keras.layers.Conv2D(16,
                            kernel_size=(3, 3),
                            padding="same",
                            activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        
        keras.layers.Conv2D(32,
                            kernel_size=(3, 3), 
                            padding="same", 
                            activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        
        keras.layers.Conv2D(64,
                            kernel_size=(3, 3), 
                            padding="same",
                            activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation="linear"),
    ]
)

2022-05-01 04:08:18.200583: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-01 04:08:18.200603: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-01 04:08:18.200617: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (me): /proc/driver/nvidia/version does not exist
2022-05-01 04:08:18.200804: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
optimizer = keras.optimizers.Adam(0.0001)

model.compile(loss='mse',  optimizer=optimizer,  metrics=['mae'])

In [12]:
model.fit(X_train,
          y_train,
          batch_size=2,
          validation_data=[X_test, y_test],
          epochs=500)

Epoch 1/500
10/10 [==============================] - 1s 34ms/step - loss: 2082449.6250 - mae: 1419.5836 - val_loss: 3228639.5000 - val_mae: 1756.7715
Epoch 2/500
10/10 [==============================] - 0s 23ms/step - loss: 1401765.0000 - mae: 1153.3713 - val_loss: 2084403.2500 - val_mae: 1389.9641
Epoch 3/500
10/10 [==============================] - 0s 23ms/step - loss: 647776.3750 - mae: 701.2473 - val_loss: 816753.5000 - val_mae: 792.4425
Epoch 4/500
10/10 [==============================] - 0s 24ms/step - loss: 199847.2031 - mae: 356.8109 - val_loss: 297140.7812 - val_mae: 423.3367
Epoch 5/500
10/10 [==============================] - 0s 23ms/step - loss: 193635.0156 - mae: 349.9789 - val_loss: 317996.9062 - val_mae: 408.1784
Epoch 6/500
10/10 [==============================] - 0s 23ms/step - loss: 137362.9062 - mae: 309.9703 - val_loss: 472710.1875 - val_mae: 510.3186
Epoch 7/500
10/10 [==============================] - 0s 23ms/step - loss: 143211.2812 - mae: 306.7093 - val_loss: 49

## - - Treino - -

In [13]:
print_evaluate(y_train, model.predict(X_train))

MAE: 19.526373291015624
MSE: 591.4110487349332
RMSE: 24.318944235614612
R2 Square 0.9902729449834226


## - - Teste - -

In [14]:
print_evaluate(y_test, model.predict(X_test))

MAE: 629.3229736328125
MSE: 516730.7703458816
RMSE: 718.8398780993452
R2 Square -2.650641422993373
